# DP_10

Adding derived features to main dataframe: comorbidities and scores

In [52]:
import pandas as pd
from IPython.display import display
import numpy as np
pd.options.display.max_columns = None
pd.options.display.max_rows = 200
import matplotlib.pyplot as plt

In [95]:
import numpy
numpy.set_printoptions(threshold=numpy.nan)

In [53]:
admissions=pd.read_csv('MAIN_DF_9.csv')
admissions=admissions.drop(['Unnamed: 0'], axis=1)
admissions=admissions.reset_index(drop=True)
print(admissions.shape)
print('unique SUBJECT_ID:', admissions.SUBJECT_ID.nunique())
print('unique HADM_ID   :', admissions.HADM_ID.nunique())
print('unique ICUSTAY_ID:', admissions.ICUSTAY_ID.nunique())
admissions.head()

(23055, 75)
unique SUBJECT_ID: 17469
unique HADM_ID   : 21650
unique ICUSTAY_ID: 23055


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,GENDER,AGE_AD,TYPE_AD,ETHNICITY,LOS,BMI,I_VEN,NI_VEN,ALBUMIN,BUN,CREATININE,GLUCOSE,HTOCRIT,LACTIC,PCO2A,PHA,PHD,PLATELETC,PO2A,SAO2,SODIUM,TBILIRUBIN,WBC,FIO2,RR,SPO2,ABPD,ABPM,ABPS,HR,NBPD,NBPM,NBPS,TEMP,PrevDays,INSMcaid,INSMcare,INSPvate,FCU_CCU,FCU_CSRU,FCU_MICU,FCU_SICU,FCU_TSICU,GCS_E_None,GCS_ESpon,GCS_EPain,GCS_ESpch,GCS_MAbFl,GCS_MAbex,GCS_MFlwd,GCS_MLoPa,GCS_MNore,GCS_MObCo,GCS_VConf,GCS_VInWo,GCS_VInso,GCS_VNORe,GCS_VNRET,GCS_VOrie,S_TEMP_Cold,S_TEMP_Cool,S_TEMP_Hot,S_TEMP_Warm,FW12,FW14,FW15,FW23,FW33,FW50,FW52,FW57,FW7
0,23,124321,234044,1,75.26000,1,1,1.1862,21.7,0,0,NaN,14.0,0.6,136.0,36.9,NaN,NaN,NaN,NaN,208.0,NaN,NaN,144.0,NaN,14.8,NaN,14.0,100.0,74.0,86.0,181.0,77.0,45.0,50.0,67.0,35.4,2.67,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0
1,34,144319,290505,1,90.89934,1,1,1.5824,27.4,0,0,NaN,36.0,1.6,116.0,35.9,NaN,NaN,NaN,NaN,343.0,NaN,NaN,139.0,NaN,7.1,60.0,14.0,98.0,NaN,NaN,NaN,72.0,66.0,78.0,112.0,36.1,0.00,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1
2,36,165660,241249,1,72.73000,0,1,4.3483,32.7,1,0,2.8,19.0,0.9,126.0,36.3,1.0,60.0,7.29,NaN,174.0,58.0,94.0,139.0,0.6,15.3,100.0,20.0,91.0,NaN,NaN,NaN,94.0,72.0,84.0,122.0,37.4,1.81,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0
3,85,112077,291697,1,76.85000,1,1,1.9909,31.8,0,0,NaN,17.0,1.4,114.0,26.4,NaN,NaN,NaN,NaN,24.0,NaN,NaN,137.0,NaN,8.3,NaN,39.0,99.0,NaN,NaN,NaN,136.0,87.0,99.0,166.0,37.0,0.00,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0
4,107,182383,252542,1,69.66000,1,0,1.0806,31.3,0,0,NaN,36.0,5.8,125.0,40.3,NaN,NaN,NaN,NaN,303.0,NaN,NaN,139.0,NaN,14.2,NaN,19.0,97.0,NaN,NaN,NaN,64.0,67.0,88.0,155.0,36.7,0.00,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0


In [54]:
comorbidities=pd.read_csv('data/MV/Comorbidities/ELIXHAUSER_AHRQ_NO_DRG_ALL_ICD.csv')
comorbidities=comorbidities.drop(['Unnamed: 0'], axis=1)
print(comorbidities.shape)
print('unique SUBJECT_ID:', comorbidities.subject_id.nunique())
print('unique hadmin:', comorbidities.hadm_id.nunique())
comorbidities=comorbidities.sort_values(by=['hadm_id'])
comorbidities=comorbidities.reset_index(drop=True)
comorbidities.head()

(58976, 32)
unique SUBJECT_ID: 46520
unique hadmin: 58976


,subject_id,hadm_id,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,hypertension,paralysis,other_neurological,chronic_pulmonary,diabetes_uncomplicated,diabetes_complicated,hypothyroidism,renal_failure,liver_disease,peptic_ulcer,aids,lymphoma,metastatic_cancer,solid_tumor,rheumatoid_arthritis,coagulopathy,obesity,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression
0,58526,100001,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,54610,100003,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,9895,100006,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1
3,23018,100007,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,533,100009,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0


In [55]:
scoresComorbidities=pd.read_csv('data/MV/Comorbidities/ELIXHAUSER_AHRQ_AHRQ_SCORE.csv')
scoresComorbidities=scoresComorbidities.drop(['Unnamed: 0'], axis=1)
print(scoresComorbidities.shape)
print('unique SUBJECT_ID:', scoresComorbidities.subject_id.nunique())
print('unique hadmin:', scoresComorbidities.hadm_id.nunique())
scoresComorbidities=scoresComorbidities.sort_values(by=['hadm_id'])
scoresComorbidities=scoresComorbidities.reset_index(drop=True)
scoresComorbidities.head()

(58976, 5)
unique SUBJECT_ID: 46520
unique hadmin: 58976


,subject_id,hadm_id,elixhauser_vanwalraven,elixhauser_sid29,elixhauser_sid30
0,58526,100001,5,5,5
1,54610,100003,11,5,7
2,9895,100006,11,11,22
3,23018,100007,0,0,0
4,533,100009,-4,-1,-1


In [56]:
scoresComorbidities=pd.merge(comorbidities, scoresComorbidities, on=['subject_id', 'hadm_id'])
scoresComorbidities=scoresComorbidities.rename({'subject_id':'SUBJECT_ID', 'hadm_id':'HADM_ID'}, axis='columns')
scoresComorbidities.shape

(58976, 35)

In [57]:
scoresComorbidities.columns

Index(['SUBJECT_ID', 'HADM_ID', 'congestive_heart_failure',
       'cardiac_arrhythmias', 'valvular_disease', 'pulmonary_circulation',
       'peripheral_vascular', 'hypertension', 'paralysis',
       'other_neurological', 'chronic_pulmonary', 'diabetes_uncomplicated',
       'diabetes_complicated', 'hypothyroidism', 'renal_failure',
       'liver_disease', 'peptic_ulcer', 'aids', 'lymphoma',
       'metastatic_cancer', 'solid_tumor', 'rheumatoid_arthritis',
       'coagulopathy', 'obesity', 'weight_loss', 'fluid_electrolyte',
       'blood_loss_anemia', 'deficiency_anemias', 'alcohol_abuse',
       'drug_abuse', 'psychoses', 'depression', 'elixhauser_vanwalraven',
       'elixhauser_sid29', 'elixhauser_sid30'],
      dtype='object')

In [58]:
scoresComorbidities=scoresComorbidities[['SUBJECT_ID', 'HADM_ID', 'congestive_heart_failure',
       'cardiac_arrhythmias', 'valvular_disease', 'pulmonary_circulation',
       'peripheral_vascular', 'hypertension', 'paralysis',
       'other_neurological', 'chronic_pulmonary', 'diabetes_uncomplicated',
       'diabetes_complicated', 'hypothyroidism', 'renal_failure',
       'liver_disease', 'peptic_ulcer', 'aids', 'lymphoma',
       'metastatic_cancer', 'solid_tumor', 'rheumatoid_arthritis',
       'coagulopathy', 'obesity', 'weight_loss', 'fluid_electrolyte',
       'blood_loss_anemia', 'deficiency_anemias', 'alcohol_abuse',
       'drug_abuse', 'psychoses', 'depression', 'elixhauser_vanwalraven',
       'elixhauser_sid29', 'elixhauser_sid30']]

In [59]:
admissions=pd.merge(admissions, scoresComorbidities, on=['SUBJECT_ID', 'HADM_ID'])
admissions.shape

(23055, 108)

In [60]:
admissions.hypertension.value_counts()

1    14068
0     8987
Name: hypertension, dtype: int64

In [61]:
APSIII=pd.read_csv('data/MV/SeverityScores/APSIII.csv')
APSIII=APSIII.drop(['Unnamed: 0'], axis=1)
print(APSIII.shape)
print('unique SUBJECT_ID:', APSIII.subject_id.nunique())
print('unique hadmin:', APSIII.hadm_id.nunique())
#APSIII=APSIII.sort_values(by=['subject_id'])
#APSIII=APSIII.reset_index(drop=True)
APSIII.head()

(61532, 21)
unique SUBJECT_ID: 46476
unique hadmin: 57786


,subject_id,hadm_id,icustay_id,apsiii,apsiii_prob,hr_score,meanbp_score,temp_score,resprate_score,pao2_aado2_score,hematocrit_score,wbc_score,creatinine_score,uo_score,bun_score,sodium_score,albumin_score,bilirubin_score,glucose_score,acidbase_score,gcs_score
0,55973,152234,200001,63,0.188678,7.0,7.0,0.0,6.0,0.0,3.0,5.0,7.0,15.0,12.0,0.0,NaN,0.0,0.0,1.0,0.0
1,27513,163557,200003,48,0.102705,7.0,10.0,0.0,9.0,NaN,3.0,5.0,0.0,0.0,7.0,0.0,0.0,6.0,0.0,1.0,0.0
2,10950,189514,200006,23,0.033927,0.0,7.0,0.0,6.0,NaN,3.0,0.0,0.0,4.0,0.0,0.0,NaN,NaN,3.0,NaN,0.0
3,20707,129310,200007,33,0.053332,1.0,15.0,0.0,6.0,NaN,3.0,0.0,0.0,5.0,0.0,0.0,0.0,NaN,3.0,NaN,0.0
4,29904,129607,200009,26,0.038894,1.0,7.0,8.0,0.0,0.0,3.0,0.0,0.0,4.0,0.0,0.0,NaN,NaN,0.0,3.0,0.0


In [62]:
APSIII.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'apsiii', 'apsiii_prob',
       'hr_score', 'meanbp_score', 'temp_score', 'resprate_score',
       'pao2_aado2_score', 'hematocrit_score', 'wbc_score', 'creatinine_score',
       'uo_score', 'bun_score', 'sodium_score', 'albumin_score',
       'bilirubin_score', 'glucose_score', 'acidbase_score', 'gcs_score'],
      dtype='object')

In [63]:
APSIII.isnull().sum()

subject_id              0
hadm_id                 0
icustay_id              0
apsiii                  0
apsiii_prob             0
hr_score             2484
meanbp_score         9244
temp_score          10150
resprate_score       9303
pao2_aado2_score    41172
hematocrit_score     1992
wbc_score            2362
creatinine_score     8632
uo_score             8331
bun_score            8642
sodium_score         6728
albumin_score       42207
bilirubin_score     35426
glucose_score        7080
acidbase_score      27159
gcs_score           10076
dtype: int64

In [64]:
APSIII=APSIII[['subject_id', 'hadm_id', 'icustay_id', 'apsiii']]

In [65]:
APSIII=APSIII.rename({'subject_id':'SUBJECT_ID', 'hadm_id':'HADM_ID',
                     'icustay_id': 'ICUSTAY_ID'}, axis='columns')

In [66]:
admissions=pd.merge(admissions, APSIII, on=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'])
admissions.shape

(23055, 109)

In [67]:
OASIS=pd.read_csv('data/MV/SeverityScores/OASIS.csv')
OASIS=OASIS.drop(['Unnamed: 0'], axis=1)
print(OASIS.shape)
print('unique SUBJECT_ID:', OASIS.subject_id.nunique())
print('unique hadmin:', OASIS.hadm_id.nunique())
#OASIS=OASIS.sort_values(by=['subject_id'])
#OASIS=OASIS.reset_index(drop=True)
OASIS.head()

(61532, 28)
unique SUBJECT_ID: 46476
unique hadmin: 57786


,subject_id,hadm_id,icustay_id,icustay_age_group,hospital_expire_flag,icustay_expire_flag,oasis,oasis_prob,age,age_score,preiculos,preiculos_score,gcs,gcs_score,heartrate,heartrate_score,meanbp,meanbp_score,resprate,resprate_score,temp,temp_score,urineoutput,urineoutput_score,mechvent,mechvent_score,electivesurgery,electivesurgery_score
0,55973,152234,200001,adult,0,0,42,0.305849,61,6,7 days 03:02:12,1,14.0,3.0,134.0,6.0,60.000000,2.0,32.0,6.0,36.388889,2.0,250.0,10.0,0,0,0,6
1,27513,163557,200003,adult,0,0,35,0.152892,48,3,02:48:04,3,15.0,0.0,122.0,3.0,179.000000,3.0,39.0,6.0,36.388889,2.0,3652.0,0.0,1,9,0,6
2,10950,189514,200006,adult,0,0,32,0.109623,54,6,00:01:14,5,15.0,0.0,73.0,0.0,61.000000,2.0,27.0,1.0,36.166666,2.0,1955.0,1.0,1,9,0,6
3,20707,129310,200007,adult,0,0,26,0.054187,43,3,00:01:37,5,15.0,0.0,104.0,1.0,50.666698,3.0,29.0,1.0,36.388889,2.0,1295.0,5.0,0,0,0,6
4,29904,129607,200009,adult,0,0,25,0.048012,47,3,-00:10:28,5,15.0,0.0,106.0,1.0,60.000000,2.0,17.5,0.0,34.599998,4.0,1570.0,1.0,1,9,1,0


In [68]:
OASIS.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'icustay_age_group',
       'hospital_expire_flag', 'icustay_expire_flag', 'oasis', 'oasis_prob',
       'age', 'age_score', 'preiculos', 'preiculos_score', 'gcs', 'gcs_score',
       'heartrate', 'heartrate_score', 'meanbp', 'meanbp_score', 'resprate',
       'resprate_score', 'temp', 'temp_score', 'urineoutput',
       'urineoutput_score', 'mechvent', 'mechvent_score', 'electivesurgery',
       'electivesurgery_score'],
      dtype='object')

In [69]:
OASIS.isnull().sum()

subject_id                   0
hadm_id                      0
icustay_id                   0
icustay_age_group            0
hospital_expire_flag         0
icustay_expire_flag          0
oasis                        0
oasis_prob                   0
age                          0
age_score                    0
preiculos                    0
preiculos_score              0
gcs                       9389
gcs_score                 9389
heartrate                 2484
heartrate_score           2484
meanbp                    9244
meanbp_score              9244
resprate                  9303
resprate_score            9303
temp                     10150
temp_score               10150
urineoutput               8331
urineoutput_score         8331
mechvent                     0
mechvent_score               0
electivesurgery              0
electivesurgery_score        0
dtype: int64

In [70]:
OASIS=OASIS[['subject_id', 'hadm_id', 'icustay_id',
       'icustay_expire_flag', 'oasis',
       'gcs', 'urineoutput',
       'electivesurgery']]

In [71]:
OASIS=OASIS.rename({'subject_id':'SUBJECT_ID', 'hadm_id':'HADM_ID',
                     'icustay_id': 'ICUSTAY_ID'}, axis='columns')

In [72]:
admissions=pd.merge(admissions, OASIS, on=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'])
admissions.shape

(23055, 114)

In [73]:
SAPS=pd.read_csv('data/MV/SeverityScores/SAPS.csv')
SAPS=SAPS.drop(['Unnamed: 0'], axis=1)
print(SAPS.shape)
print('unique SUBJECT_ID:', SAPS.subject_id.nunique())
print('unique hadmin:', SAPS.hadm_id.nunique())
#SAPS=SAPS.sort_values(by=['subject_id'])
#SAPS=SAPS.reset_index(drop=True)
SAPS.head()

(61532, 19)
unique SUBJECT_ID: 46476
unique hadmin: 57786


,subject_id,hadm_id,icustay_id,saps,age_score,hr_score,sysbp_score,resp_score,temp_score,uo_score,vent_score,bun_score,hematocrit_score,wbc_score,glucose_score,potassium_score,sodium_score,bicarbonate_score,gcs_score
0,55973,152234,200001,20,2,2.0,0.0,1.0,0.0,3.0,0,4.0,2.0,2.0,4.0,0.0,0.0,0.0,0.0
1,27513,163557,200003,24,1,2.0,2.0,3.0,1.0,1.0,3,1.0,0.0,4.0,4.0,1.0,0.0,1.0,0.0
2,10950,189514,200006,15,1,2.0,0.0,1.0,0.0,0.0,3,1.0,2.0,0.0,4.0,0.0,0.0,1.0,0.0
3,20707,129310,200007,6,0,0.0,0.0,1.0,0.0,0.0,0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
4,29904,129607,200009,13,1,0.0,0.0,0.0,1.0,0.0,3,1.0,2.0,0.0,4.0,1.0,0.0,0.0,0.0


In [74]:
SAPS.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'saps', 'age_score', 'hr_score',
       'sysbp_score', 'resp_score', 'temp_score', 'uo_score', 'vent_score',
       'bun_score', 'hematocrit_score', 'wbc_score', 'glucose_score',
       'potassium_score', 'sodium_score', 'bicarbonate_score', 'gcs_score'],
      dtype='object')

In [75]:
SAPS.isnull().sum()

subject_id               0
hadm_id                  0
icustay_id               0
saps                     0
age_score                0
hr_score              2484
sysbp_score           9278
resp_score            9303
temp_score           10150
uo_score              8331
vent_score               0
bun_score             8642
hematocrit_score      1992
wbc_score             2362
glucose_score         7080
potassium_score       6661
sodium_score          6728
bicarbonate_score     7121
gcs_score             9389
dtype: int64

In [76]:
SAPS=SAPS[['subject_id', 'hadm_id', 'icustay_id', 'saps']]

In [77]:
SAPS=SAPS.rename({'subject_id':'SUBJECT_ID', 'hadm_id':'HADM_ID',
                     'icustay_id': 'ICUSTAY_ID'}, axis='columns')

In [78]:
admissions=pd.merge(admissions, SAPS, on=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'])
admissions.shape

(23055, 115)

In [79]:
SAPSII=pd.read_csv('data/MV/SeverityScores/SAPSII.csv')
SAPSII=SAPSII.drop(['Unnamed: 0'], axis=1)
print(SAPSII.shape)
print('unique SUBJECT_ID:', SAPSII.subject_id.nunique())
print('unique hadmin:', SAPSII.hadm_id.nunique())
#SAPSII=SAPSII.sort_values(by=['hadm_id'])
#SAPSII=SAPSII.reset_index(drop=True)
SAPSII.head()

(61532, 20)
unique SUBJECT_ID: 46476
unique hadmin: 57786


,subject_id,hadm_id,icustay_id,sapsii,sapsii_prob,age_score,hr_score,sysbp_score,temp_score,pao2fio2_score,uo_score,bun_score,wbc_score,potassium_score,sodium_score,bicarbonate_score,bilirubin_score,gcs_score,comorbidity_score,admissiontype_score
0,55973,152234,200001,38,0.212560,12,4.0,5.0,0.0,NaN,11.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,6
1,27513,163557,200003,30,0.106398,7,4.0,5.0,0.0,NaN,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,0,8
2,10950,189514,200006,20,0.037205,7,2.0,5.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0,6
3,20707,129310,200007,18,0.029295,7,0.0,5.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0,6
4,29904,129607,200009,21,0.041754,7,0.0,5.0,0.0,6.0,0.0,0.0,0.0,3.0,0.0,0.0,NaN,0.0,0,0


In [80]:
SAPSII.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'sapsii', 'sapsii_prob',
       'age_score', 'hr_score', 'sysbp_score', 'temp_score', 'pao2fio2_score',
       'uo_score', 'bun_score', 'wbc_score', 'potassium_score', 'sodium_score',
       'bicarbonate_score', 'bilirubin_score', 'gcs_score',
       'comorbidity_score', 'admissiontype_score'],
      dtype='object')

In [81]:
SAPSII.isnull().sum()

subject_id                 0
hadm_id                    0
icustay_id                 0
sapsii                     0
sapsii_prob                0
age_score                  0
hr_score                2484
sysbp_score             9278
temp_score             10150
pao2fio2_score         41667
uo_score                8331
bun_score               8713
wbc_score               2362
potassium_score         6661
sodium_score            6728
bicarbonate_score       7121
bilirubin_score        35426
gcs_score               9389
comorbidity_score          0
admissiontype_score        0
dtype: int64

In [82]:
SAPSII=SAPSII[['subject_id', 'hadm_id', 'icustay_id', 'sapsii',
       'potassium_score','bicarbonate_score',
       'comorbidity_score']]

In [83]:
SAPSII=SAPSII.rename({'subject_id':'SUBJECT_ID', 'hadm_id':'HADM_ID',
                     'icustay_id': 'ICUSTAY_ID'}, axis='columns')

In [84]:
admissions=pd.merge(admissions, SAPSII, on=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'])
admissions.shape

(23055, 119)

In [85]:
SOFA=pd.read_csv('data/MV/SeverityScores/SOFA.csv')
SOFA=SOFA.drop(['Unnamed: 0'], axis=1)
print(SOFA.shape)
print('unique SUBJECT_ID:', SOFA.subject_id.nunique())
print('unique hadmin:', SOFA.hadm_id.nunique())
#SOFA=SOFA.sort_values(by=['subject_id'])
#SOFA=SOFA.reset_index(drop=True)
SOFA.head()

(61532, 10)
unique SUBJECT_ID: 46476
unique hadmin: 57786


,subject_id,hadm_id,icustay_id,sofa,respiration,coagulation,liver,cardiovascular,cns,renal
0,55973,152234,200001,6,NaN,1.0,0.0,1.0,1.0,3.0
1,27513,163557,200003,6,2.0,1.0,2.0,1.0,0.0,0.0
2,10950,189514,200006,1,NaN,0.0,NaN,1.0,0.0,0.0
3,20707,129310,200007,1,NaN,0.0,NaN,1.0,0.0,0.0
4,29904,129607,200009,3,0.0,2.0,NaN,1.0,0.0,0.0


In [86]:
SOFA.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'sofa', 'respiration',
       'coagulation', 'liver', 'cardiovascular', 'cns', 'renal'],
      dtype='object')

In [87]:
SOFA.isnull().sum()

subject_id            0
hadm_id               0
icustay_id            0
sofa                  0
respiration       36628
coagulation        2293
liver             35426
cardiovascular     9040
cns                9389
renal              5038
dtype: int64

In [88]:
SOFA=SOFA[['subject_id', 'hadm_id', 'icustay_id', 'sofa',
       'coagulation', 'liver', 'cardiovascular', 'cns', 'renal']]

In [89]:
SOFA=SOFA.rename({'subject_id':'SUBJECT_ID', 'hadm_id':'HADM_ID',
                     'icustay_id': 'ICUSTAY_ID'}, axis='columns')

In [90]:
admissions=pd.merge(admissions, SOFA, on=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'])
admissions.shape

(23055, 125)

In [97]:
admissions.columns

Index(['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'GENDER', 'AGE_AD', 'TYPE_AD',
       'ETHNICITY', 'LOS', 'BMI', 'I_VEN',
       ...
       'sapsii', 'potassium_score', 'bicarbonate_score', 'comorbidity_score',
       'sofa', 'coagulation', 'liver', 'cardiovascular', 'cns', 'renal'],
      dtype='object', length=125)

In [98]:
admissions.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,GENDER,AGE_AD,TYPE_AD,ETHNICITY,LOS,BMI,I_VEN,NI_VEN,ALBUMIN,BUN,CREATININE,GLUCOSE,HTOCRIT,LACTIC,PCO2A,PHA,PHD,PLATELETC,PO2A,SAO2,SODIUM,TBILIRUBIN,WBC,FIO2,RR,SPO2,ABPD,ABPM,ABPS,HR,NBPD,NBPM,NBPS,TEMP,PrevDays,INSMcaid,INSMcare,INSPvate,FCU_CCU,FCU_CSRU,FCU_MICU,FCU_SICU,FCU_TSICU,GCS_E_None,GCS_ESpon,GCS_EPain,GCS_ESpch,GCS_MAbFl,GCS_MAbex,GCS_MFlwd,GCS_MLoPa,GCS_MNore,GCS_MObCo,GCS_VConf,GCS_VInWo,GCS_VInso,GCS_VNORe,GCS_VNRET,GCS_VOrie,S_TEMP_Cold,S_TEMP_Cool,S_TEMP_Hot,S_TEMP_Warm,FW12,FW14,FW15,FW23,FW33,FW50,FW52,FW57,FW7,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,hypertension,paralysis,other_neurological,chronic_pulmonary,diabetes_uncomplicated,diabetes_complicated,hypothyroidism,renal_failure,liver_disease,peptic_ulcer,aids,lymphoma,metastatic_cancer,solid_tumor,rheumatoid_arthritis,coagulopathy,obesity,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression,elixhauser_vanwalraven,elixhauser_sid29,elixhauser_sid30,apsiii,icustay_expire_flag,oasis,gcs,urineoutput,electivesurgery,saps,sapsii,potassium_score,bicarbonate_score,comorbidity_score,sofa,coagulation,liver,cardiovascular,cns,renal
0,23,124321,234044,1,75.26000,1,1,1.1862,21.7,0,0,NaN,14.0,0.6,136.0,36.9,NaN,NaN,NaN,NaN,208.0,NaN,NaN,144.0,NaN,14.8,NaN,14.0,100.0,74.0,86.0,181.0,77.0,45.0,50.0,67.0,35.4,2.67,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,31,0,24,15.0,6445.0,0,19,39,0.0,0.0,0,1,0.0,NaN,1.0,0.0,0.0
1,34,144319,290505,1,90.89934,1,1,1.5824,27.4,0,0,NaN,36.0,1.6,116.0,35.9,NaN,NaN,NaN,NaN,343.0,NaN,NaN,139.0,NaN,7.1,60.0,14.0,98.0,NaN,NaN,NaN,72.0,66.0,78.0,112.0,36.1,0.00,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33,0,40,14.0,875.0,0,21,36,0.0,0.0,0,3,0.0,NaN,1.0,1.0,1.0
2,36,165660,241249,1,72.73000,0,1,4.3483,32.7,1,0,2.8,19.0,0.9,126.0,36.3,1.0,60.0,7.29,NaN,174.0,58.0,94.0,139.0,0.6,15.3,100.0,20.0,91.0,NaN,NaN,NaN,94.0,72.0,84.0,122.0,37.4,1.81,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,8,8,27,0,23,15.0,2030.0,1,14,31,0.0,0.0,0,5,1.0,NaN,1.0,0.0,0.0
3,85,112077,291697,1,76.85000,1,1,1.9909,31.8,0,0,NaN,17.0,1.4,114.0,26.4,NaN,NaN,NaN,NaN,24.0,NaN,NaN,137.0,NaN,8.3,NaN,39.0,99.0,NaN,NaN,NaN,136.0,87.0,99.0,166.0,37.0,0.00,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,7,6,15,47,0,35,15.0,1650.0,0,19,34,0.0,3.0,0,5,3.0,NaN,1.0,0.0,1.0
4,107,182383,252542,1,69.66000,1,0,1.0806,31.3,0,0,NaN,36.0,5.8,125.0,40.3,NaN,NaN,NaN,NaN,303.0,NaN,NaN,139.0,NaN,14.2,NaN,19.0,97.0,NaN,NaN,NaN,64.0,67.0,88.0,155.0,36.7,0.00,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,6,14,15,43,0,23,15.0,NaN,0,21,42,3.0,0.0,0,5,0.0,NaN,1.0,0.0,4.0


In [99]:
admissions=admissions.rename({'congestive_heart_failure':'hfail', 'cardiac_arrhythmias': 'carr',
                             'valvular_disease': 'vdis', 'pulmonary_circulation': 'pcirc', 
                             'peripheral_vascular': 'pvasc', 'hypertension': 'htens',
                             'other_neurological': 'oneur', 'chronic_pulmonary': 'chpul',
                             'diabetes_uncomplicated': 'dunc', 'diabetes_complicated': 'dcom',
                             'hypothyroidism': 'hthy', 'renal_failure': 'rfail',
                             'liver_disease': 'ldis', 'peptic_ulcer': 'pulc',
                             'metastatic_cancer': 'mcancer', 'rheumatoid_arthritis': 'rhart',
                             'coagulopathy': 'coath', 'weight_loss': 'wlost',
                             'fluid_electrolyte': 'flele', 'blood_loss_anemia': 'blane',
                             'deficiency_anemias': 'defane', 'alcohol_abuse': 'alch',
                             'drug_abuse': 'drg', 'elixhauser_vanwalraven': 'elixvw',
                             'elixhauser_sid29': 'elix29', 'elixhauser_sid30': 'elix30',
                             'icustay_expire_flag': 'icu_flag', 'urineoutput': 'uo',
                             'electivesurgery': 'surg', 'potassium_score': 'potass',
                             'bicarbonate_score': 'bicsc', 'comorbidity_score': 'comsco'}, axis='columns')

In [100]:
admissions.to_csv('MAIN_DF_10.csv')